In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from config import FILENAME, DATECOLUMN, DATEFORMAT, TIME_DIFF_SIZE, TIME_DIFF_UNIT, GROUPBY_COL, MAX_SESSION_BREAK_SECS, MAX_LEVEL, STD_OUTLIER

In [2]:
def load_data(filename: str,
              datecolumn: str,
              dateformat: str) -> pd.DataFrame:
    return (
        pd.read_csv(filename)
        .rename(str.strip, axis='columns')
        .assign(
            **{f'{datecolumn}' : lambda df:
                pd.to_datetime(df[datecolumn],
                            format=(DATEFORMAT)
                )
        })
        .sort_values(datecolumn)
)

wowah: pd.DataFrame = load_data(FILENAME, DATECOLUMN, DATEFORMAT)
wowah
    


,char,level,race,charclass,zone,guild,timestamp
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09
...,...,...,...,...,...,...,...
10826711,88004,80,Blood Elf,Death Knight,Icecrown,135,2008-12-31 23:50:18
10826712,86517,80,Blood Elf,Death Knight,Undercity,115,2008-12-31 23:50:18
10826713,86679,80,Blood Elf,Death Knight,Eastern Plaguelands,459,2008-12-31 23:50:18
10826715,86851,80,Blood Elf,Death Knight,Halls of Stone,103,2008-12-31 23:50:18


In [3]:
def max_amount_per_group(df: pd.DataFrame,
                         groupby_col: str,
                         column) -> int:
    return df.groupby(groupby_col)[column].nunique().max()

groupby_col = 'char'
columns = wowah.columns

for col in columns:
    print(f"Max amount of unique values per '{groupby_col}' in '{col}': {max_amount_per_group(wowah, groupby_col, col)}")

Max amount of unique values per 'char' in 'char': 1
Max amount of unique values per 'char' in 'level': 80
Max amount of unique values per 'char' in 'race': 5
Max amount of unique values per 'char' in 'charclass': 8
Max amount of unique values per 'char' in 'zone': 145
Max amount of unique values per 'char' in 'guild': 10
Max amount of unique values per 'char' in 'timestamp': 42770


In [4]:
def add_time_diff(df: pd.DataFrame,
                            groupby_col: str,
                            date_col: str,
                            time_diff_size: int = 1,
                            time_diff_unit: str = 's') -> pd.DataFrame:
    return (
        wowah
        .assign(
            time_diff = lambda df:
                df.groupby(groupby_col)[date_col]
                .diff()
                / np.timedelta64(time_diff_size, time_diff_unit)
        )
    )

wowah_time_diff: pd.DataFrame = add_time_diff(wowah, GROUPBY_COL, DATECOLUMN)
wowah_time_diff

,char,level,race,charclass,zone,guild,timestamp,time_diff
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04,NaN
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04,NaN
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04,NaN
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04,NaN
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09,NaN
...,...,...,...,...,...,...,...,...
10826711,88004,80,Blood Elf,Death Knight,Icecrown,135,2008-12-31 23:50:18,703.0
10826712,86517,80,Blood Elf,Death Knight,Undercity,115,2008-12-31 23:50:18,703.0
10826713,86679,80,Blood Elf,Death Knight,Eastern Plaguelands,459,2008-12-31 23:50:18,91298.0
10826715,86851,80,Blood Elf,Death Knight,Halls of Stone,103,2008-12-31 23:50:18,703.0


**Explore categorial variables vs outlier or not**

**Add session indicator**

In [5]:
print("Median session time:")
(
    wowah_time_diff
    .loc[: , ['time_diff']]
    .dropna()
    .median()
)

Median session time:


time_diff    613.0
dtype: float64

In [6]:
def add_sessions(df: pd.DataFrame,
                 max_session_break: int) -> pd.DataFrame:
    return (
        df
        .assign(new_session = lambda df:
            df['time_diff'] > max_session_break)
        .assign(session = lambda df:
            df.groupby('char')['new_session']
            .cumsum()
        )
        .assign(time_diff_within_session = lambda df:
            np.where(df['new_session'], 0, df['time_diff'])
        )
        .assign(time_diff_between_session = lambda df:
            np.where(df['new_session'], df['time_diff'], np.nan)
        )
    )

wowah_sessions = add_sessions(wowah_time_diff, MAX_SESSION_BREAK_SECS)
wowah_sessions

,char,level,race,charclass,zone,guild,timestamp,time_diff,new_session,session,time_diff_within_session,time_diff_between_session
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04,NaN,False,0,NaN,NaN
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09,NaN,False,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
10826711,88004,80,Blood Elf,Death Knight,Icecrown,135,2008-12-31 23:50:18,703.0,False,93,703.0,NaN
10826712,86517,80,Blood Elf,Death Knight,Undercity,115,2008-12-31 23:50:18,703.0,False,148,703.0,NaN
10826713,86679,80,Blood Elf,Death Knight,Eastern Plaguelands,459,2008-12-31 23:50:18,91298.0,True,106,0.0,91298.0
10826715,86851,80,Blood Elf,Death Knight,Halls of Stone,103,2008-12-31 23:50:18,703.0,False,113,703.0,NaN


**Play time to get to level 60**

In [24]:
def calc_play_time_to_level(df: pd.DataFrame,
                            level: int) -> pd.DataFrame:
    return (
        df
        .groupby(['char','level'])
        ['time_diff_within_session']
        .sum()
        .groupby('level')
        .mean()
        .loc[:level]
        .sum()
        /3600
    )
calc_play_time_to_level(wowah_sessions,
                        MAX_LEVEL)

154.4428769058169

In [7]:
def calc_play_time_to_level(df: pd.DataFrame,
                            level: int) -> pd.DataFrame:
    return (
        df
        .assign(
            session_length = lambda df:
                df.groupby(['char', 'session'])['time_diff_within_session']
                .transform(sum)
        )
        .assign(
            **{f'time_to_level_{level}': lambda df:
                df.groupby(['char', 'session'])
                ['session_length']
                .transform(sum)
            }
        )
    )
wowah_max_level = calc_play_time_to_level(wowah_sessions, MAX_LEVEL)
wowah_max_level

,char,level,race,charclass,zone,guild,timestamp,time_diff,new_session,session,time_diff_within_session,time_diff_between_session,session_length,time_to_level_60
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,588.0,1176.0
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,5419.0,54190.0
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,1801.0,7204.0
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,11996.0,239920.0
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09,NaN,False,0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10826711,88004,80,Blood Elf,Death Knight,Icecrown,135,2008-12-31 23:50:18,703.0,False,93,703.0,NaN,11736.0,164304.0
10826712,86517,80,Blood Elf,Death Knight,Undercity,115,2008-12-31 23:50:18,703.0,False,148,703.0,NaN,36918.0,1550556.0
10826713,86679,80,Blood Elf,Death Knight,Eastern Plaguelands,459,2008-12-31 23:50:18,91298.0,True,106,0.0,91298.0,0.0,0.0
10826715,86851,80,Blood Elf,Death Knight,Halls of Stone,103,2008-12-31 23:50:18,703.0,False,113,703.0,NaN,5461.0,38227.0


In [8]:
print('Median time to get to level 60')
(
    wowah_max_level
    .loc[:, [f'time_to_level_{MAX_LEVEL}']]
    .median()
    / 3600
)

Median time to get to level 60


time_to_level_60    57.0
dtype: float64

**Current churn rate**

In [9]:
def add_churn_statistics(df: pd.DataFrame,
                        std_diff: int = 3) -> pd.DataFrame:
        return (
                df
                .assign(time_diff_mean_per_char = lambda df:
                df.groupby(groupby_col)['time_diff_between_session']
                .transform('mean')
                )
                .assign(time_diff_std_per_char = lambda df:
                df.groupby(groupby_col)['time_diff_between_session']
                .transform('std')
                )
                .assign(time_diff_session_outlier = lambda df:
                df['time_diff_between_session'] > std_diff * df['time_diff_std_per_char']
                )
        )

wowah_churn = add_churn_statistics(wowah_max_level, STD_OUTLIER)
wowah_churn

,char,level,race,charclass,zone,guild,timestamp,time_diff,new_session,session,time_diff_within_session,time_diff_between_session,session_length,time_to_level_60,time_diff_mean_per_char,time_diff_std_per_char,time_diff_session_outlier
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,588.0,1176.0,555128.000000,1.061367e+06,False
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,5419.0,54190.0,84063.442509,1.519825e+05,False
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,1801.0,7204.0,80993.000000,8.486837e+04,False
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,11996.0,239920.0,85752.583333,1.481930e+05,False
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09,NaN,False,0,NaN,NaN,0.0,0.0,665909.319149,1.104473e+06,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10826711,88004,80,Blood Elf,Death Knight,Icecrown,135,2008-12-31 23:50:18,703.0,False,93,703.0,NaN,11736.0,164304.0,28602.247312,3.578155e+04,False
10826712,86517,80,Blood Elf,Death Knight,Undercity,115,2008-12-31 23:50:18,703.0,False,148,703.0,NaN,36918.0,1550556.0,20822.310811,4.811883e+04,False
10826713,86679,80,Blood Elf,Death Knight,Eastern Plaguelands,459,2008-12-31 23:50:18,91298.0,True,106,0.0,91298.0,0.0,0.0,30202.066038,3.256262e+04,False
10826715,86851,80,Blood Elf,Death Knight,Halls of Stone,103,2008-12-31 23:50:18,703.0,False,113,703.0,NaN,5461.0,38227.0,28004.566372,3.885484e+04,False


In [10]:
print("Ratio of new sessions that count as an outlier and thus churn:")
(
    wowah_churn
    .loc[lambda df: df['new_session']]
    .loc[:, ['time_diff_session_outlier']]
    .mean()
)

Ratio of new sessions that count as an outlier and thus churn:


time_diff_session_outlier    0.026816
dtype: float64

**Does reaching max level have any influence on churn?**

In [11]:
(
    wowah_churn
    .loc[lambda df: df['new_session'] == True]
    .loc[lambda df: df['level'] <= MAX_LEVEL]
    .assign(max_level = lambda df:
        df['level'] == MAX_LEVEL
    )
    .groupby('max_level')
    ['time_diff_session_outlier']
    .value_counts(normalize=True)
)

max_level  time_diff_session_outlier
False      False                        0.963493
           True                         0.036507
True       False                        0.966790
           True                         0.033210
Name: time_diff_session_outlier, dtype: float64

In [15]:
def process_dataframe(filename:str) -> pd.DataFrame:
    return (
        load_data(filename=filename,
                  datecolumn=DATECOLUMN,
                  dateformat=DATEFORMAT
        )
        .pipe(add_time_diff,
              groupby_col='char',
              date_col=DATECOLUMN,
              time_diff_size=TIME_DIFF_SIZE,
              time_diff_unit=TIME_DIFF_UNIT
        )
        .pipe(add_sessions,
              max_session_break=MAX_SESSION_BREAK_SECS
        )
        .pipe(add_churn_statistics,
              std_diff=STD_OUTLIER)
    )

wowah_processed = process_dataframe('wowah_data.csv')
wowah_processed

,char,level,race,charclass,zone,guild,timestamp,time_diff,new_session,session,time_diff_within_session,time_diff_between_session,session_length,time_to_level_60,time_diff_mean_per_char,time_diff_std_per_char,time_diff_session_outlier
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,588.0,1176.0,555128.000000,1.061367e+06,False
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,5419.0,54190.0,84063.442509,1.519825e+05,False
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,1801.0,7204.0,80993.000000,8.486837e+04,False
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04,NaN,False,0,NaN,NaN,11996.0,239920.0,85752.583333,1.481930e+05,False
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09,NaN,False,0,NaN,NaN,0.0,0.0,665909.319149,1.104473e+06,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10826711,88004,80,Blood Elf,Death Knight,Icecrown,135,2008-12-31 23:50:18,703.0,False,93,703.0,NaN,11736.0,164304.0,28602.247312,3.578155e+04,False
10826712,86517,80,Blood Elf,Death Knight,Undercity,115,2008-12-31 23:50:18,703.0,False,148,703.0,NaN,36918.0,1550556.0,20822.310811,4.811883e+04,False
10826713,86679,80,Blood Elf,Death Knight,Eastern Plaguelands,459,2008-12-31 23:50:18,91298.0,True,106,0.0,91298.0,0.0,0.0,30202.066038,3.256262e+04,False
10826715,86851,80,Blood Elf,Death Knight,Halls of Stone,103,2008-12-31 23:50:18,703.0,False,113,703.0,NaN,5461.0,38227.0,28004.566372,3.885484e+04,False


In [16]:
from advanced_value_counts.avc import AdvancedValueCounts as AVC

avc = AVC(
    df = wowah_processed,
    column = 'race',
    groupby_col = 'time_diff_session_outlier'
)
avc.avc_df

subgroup_ratio     count  r_vs_total
time_diff_session_outlier race                                           
False                     Blood Elf        0.362995   3919287    0.362001
                          Orc              0.086134    930002    0.085899
                          Tauren           0.215362   2325288    0.214773
                          Troll            0.101788   1099013    0.101509
                          Undead           0.233721   2523506    0.233081
                          _total           1.000000  10797096    0.997263
True                      Blood Elf        0.361293     10708    0.000989
                          Orc              0.103043      3054    0.000282
                          Tauren           0.196707      5830    0.000538
                          Troll            0.114583      3396    0.000314
                          Undead           0.224374      6650    0.000614
                          _total           1.000000     29638    0.002737
_all                      Blood Elf        0.362990   3929995    0.362990
                          Orc              0.086181    933056    0.086181
                          Tauren           0.215311   2331118    0.215311
                          Troll            0.101823   1102409    0.101823
                          Undead           0.233695   2530156    0.233695
                          _total           1.000000  10826734    1.000000

In [17]:
avc.column = 'charclass'
avc.avc_df

subgroup_ratio     count  r_vs_total
time_diff_session_outlier charclass                                         
False                     Death Knight        0.021184    228729    0.021126
                          Druid               0.084654    914014    0.084422
                          Hunter              0.143164   1545755    0.142772
                          Mage                0.132813   1433996    0.132450
                          Paladin             0.107046   1155785    0.106753
                          Priest              0.104099   1123967    0.103814
                          Rogue               0.096993   1047240    0.096727
                          Shaman              0.067121    724717    0.066938
                          Warlock             0.124459   1343798    0.124119
                          Warrior             0.118467   1279095    0.118142
                          _total              1.000000  10797096    0.997263
True                      Death Knight        0.045887      1360    0.000126
                          Druid               0.078615      2330    0.000215
                          Hunter              0.140664      4169    0.000385
                          Mage                0.132195      3918    0.000362
                          Paladin             0.096734      2867    0.000265
                          Priest              0.104056      3084    0.000285
                          Rogue               0.097645      2894    0.000267
                          Shaman              0.066199      1962    0.000181
                          Warlock             0.121837      3611    0.000334
                          Warrior             0.116168      3443    0.000318
                          _total              1.000000     29638    0.002737
_all                      Death Knight        0.021252    230089    0.021252
                          Druid               0.084637    916344    0.084637
                          Hunter              0.143157   1549924    0.143157
                          Mage                0.132811   1437914    0.132811
                          Paladin             0.107018   1158652    0.107018
                          Priest              0.104099   1127051    0.104099
                          Rogue               0.096995   1050134    0.096995
                          Shaman              0.067119    726679    0.067119
                          Warlock             0.124452   1347409    0.124452
                          Warrior             0.118460   1282538    0.118460
                          _total              1.000000  10826734    1.000000

In [18]:
avc.column = 'zone'
avc.max_subgroups = 10
avc.avc_df

subgroup_ratio     count  \
time_diff_session_outlier zone                                           
False                     Alterac Valley            0.026085    281641   
                          Arathi Basin              0.028214    304630   
                          Hellfire Peninsula        0.032930    355550   
                          Isle of Quel'Danas        0.029110    314306   
                          Karazhan                  0.044234    477603   
                          Nagrand                   0.032760    353714   
                          Orgrimmar                 0.063242    682831   
                          Shadowmoon Valley         0.025812    278697   
                          Shattrath City            0.101763   1098749   
                          Terokkar Forest           0.035459    382859   
                          _other                    0.580389   6266516   
                          _total                    1.000000  10797096   
True                      Alterac Valley            0.001215        36   
                          Arathi Basin              0.008773       260   
                          Hellfire Peninsula        0.031817       943   
                          Isle of Quel'Danas        0.022471       666   
                          Karazhan                  0.003003        89   
                          Nagrand                   0.027802       824   
                          Orgrimmar                 0.181220      5371   
                          Shadowmoon Valley         0.009009       267   
                          Shattrath City            0.219988      6520   
                          Terokkar Forest           0.026824       795   
                          _other                    0.467879     13867   
                          _total                    1.000000     29638   
_all                      Alterac Valley            0.026017    281677   
                          Arathi Basin              0.028161    304890   
                          Hellfire Peninsula        0.032927    356493   
                          Isle of Quel'Danas        0.029092    314972   
                          Karazhan                  0.044122    477692   
                          Nagrand                   0.032747    354538   
                          Orgrimmar                 0.063565    688202   
                          Shadowmoon Valley         0.025766    278964   
                          Shattrath City            0.102087   1105269   
                          Terokkar Forest           0.035436    383654   
                          _other                    0.580081   6280383   
                          _total                    1.000000  10826734   

                                              r_vs_total  
time_diff_session_outlier zone                            
False                     Alterac Valley        0.026013  
                          Arathi Basin          0.028137  
                          Hellfire Peninsula    0.032840  
                          Isle of Quel'Danas    0.029031  
                          Karazhan              0.044113  
                          Nagrand               0.032670  
                          Orgrimmar             0.063069  
                          Shadowmoon Valley     0.025742  
                          Shattrath City        0.101485  
                          Terokkar Forest       0.035362  
                          _other                0.578800  
                          _total                0.997263  
True                      Alterac Valley        0.000003  
                          Arathi Basin          0.000024  
                          Hellfire Peninsula    0.000087  
                          Isle of Quel'Danas    0.000062  
                          Karazhan              0.000008  
                          Nagrand               0.000076  
                          Orgrimmar             0.000